## Conv1D를 활용한 다음 movie 리뷰 감성 분석

- daum_movie_review.csv 사용

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('./data/daum_movie_review.csv', sep=',')
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [8]:
print('전체 데이터 개수: {}'.format(len(df)))
data_length = df['review'].astype(str).apply(len)
data_length.head()

전체 데이터 개수: 14725


0     22
1     44
2    107
3     19
4      4
Name: review, dtype: int64

In [10]:
data_length.describe()

count    14725.000000
mean        52.780170
std         56.534392
min          2.000000
25%         16.000000
50%         34.000000
75%         70.000000
max        482.000000
Name: review, dtype: float64

In [11]:
#리뷰 통계 정보
print('리뷰 길이 최댓값: {}'.format(np.max(data_length)))
print('리뷰 길이 최솟값: {}'.format(np.min(data_length)))
print('리뷰 길이 평균값: {:.2f}'.format(np.mean(data_length)))
print('리뷰 길이 표준편차: {:.2f}'.format(np.std(data_length)))
print('리뷰 길이 중간값: {}'.format(np.median(data_length)))
print('리뷰 길이 제1사분위: {}'.format(np.percentile(data_length,25)))
print('리뷰 길이 제3사분위: {}'.format(np.percentile(data_length,75)))

리뷰 길이 최댓값: 482
리뷰 길이 최솟값: 2
리뷰 길이 평균값: 52.78
리뷰 길이 표준편차: 56.53
리뷰 길이 중간값: 34.0
리뷰 길이 제1사분위: 16.0
리뷰 길이 제3사분위: 70.0


In [46]:
df.loc[data_length.sort_values(ascending=False).index].head()

,review,rating,date,title
1225,낚였다! 일반적인 어벤저스 영화임 10주년 기념작이라고 기대하고서 예매 하면 실망....,8,2018.04.27,인피니티 워
4905,근데 귀인으로 재판 올클리어 해서 다시 인간으로 환생 됬는데. 다시 태어나고 보니 ...,8,2018.05.12,신과함께
1537,영화를 보고 나서 얼이 빠져버렸다. 이 영화로서 결론을 낸 것은 아니지 싶은데 그 ...,9,2018.04.25,인피니티 워
7669,난 여기에 잼 없고 졸다 왔고 만석이 아니라고 댓글 단 사람들 머하는 사람들인지 잘...,9,2017.12.27,신과함께
5714,신파 환타지라는 이상한 장르라고 해서 안 보려다가 1300만이 봤다길래 궁금해서 봤...,8,2018.01.18,신과함께


In [99]:
X = df["review"]
y = [1 if i >= 6 else 0 for i in df["rating"]]
pd.Series(y).value_counts()

1    11179
0     3546
Name: count, dtype: int64

In [100]:
# lable 대신 rating score가 포함되어 있음
# 1~10점으로 구성되어 있음 >> 긍/부정 레이블(target)은 자체 생성해서
# 타겟에 대한 레코드 수 확인해서 편향되지 않도록

# 23번 네이버 영화 리뷰 긍부정 분석 참고해서 진행

In [101]:
import re
import json

import konlpy
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [102]:
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):

    review_text = re.sub('[^가-힣 ㄱ-ㅎ ㅏ-ㅣ\\ㄴ]', ' ', review)
    word_review = okt.morphs(review_text, stem=True)

    if remove_stopwords :
        word_review = [tk for tk in word_review if not tk in stop_words]

    return word_review

In [103]:
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_train_review = []

In [105]:
from tqdm import tqdm

for review in tqdm(X):
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
    else :
        clean_train_review.append([]) # str이 아닌 행을 빈 리스트를 추가

clean_train_review[:4]

100%|████████████████████████████████████████████████████████████████████████████| 14725/14725 [03:31<00:00, 69.70it/s]


[['돈', '들이다', '티', '나', '지만', '보다', '내내', '하품', '만'],
 ['몰입',
  '하다',
  '없다',
  '어렵다',
  '생각',
  '하다',
  '필요없다',
  '내',
  '전투',
  '에',
  '참여',
  '듯',
  '손',
  '에',
  '땀',
  '이남'],
 ['이전',
  '작품',
  '에',
  '비다',
  '더',
  '화려하다',
  '스케일',
  '도',
  '커지다',
  '전국',
  '맛집',
  '음식',
  '을',
  '한데',
  '모으다',
  '까지는',
  '좋다',
  '걸',
  '모두',
  '하다',
  '그릇',
  '에',
  '섞다',
  '버리다',
  '듯',
  '느낌',
  '그래도',
  '다음',
  '작품',
  '을',
  '기대하다',
  '만들다'],
 ['정도', '면', '볼', '만', '하다', '하다', '있다']]

In [106]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 리뷰 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review) # 학습데이터로 단어 사전을 생성
train_sequences = tokenizer.texts_to_sequences(clean_train_review)

In [86]:
X = train_sequences.copy()


In [97]:
word_vocab = tokenizer.word_index
len(word_vocab)
MAX_SEQ_LEN = 34

12859

In [90]:
train_inputs = pad_sequences(train_sequences, maxlen = MAX_SEQ_LEN, padding= "post")
train_labels = np.array(y)

In [96]:
train_inputs[6]
train_labels[:10]

array([ 46, 213,  11,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 1])